In [1]:
%pwd

'/Users/ryandevera/data-science/tesorio/NCVX-Neural-Structural-Optimization/notebooks'

In [2]:
%cd ..

/Users/ryandevera/data-science/tesorio/NCVX-Neural-Structural-Optimization


In [3]:
import numpy as np
import matplotlib.pyplot as plt
import time
import torch
import os
import pandas as pd

# import problems to solve
import problems
import experiments
import train
import topo_api
import topo_physics
import models
import utils

from pygranso.private.getNvar import getNvarTorch
from pygranso.pygranso import pygranso
from pygranso.pygransoStruct import pygransoStruct

from scipy.ndimage import gaussian_filter

# Running neural structural optimization with PyGranso 🧨

To check the validity of our new `torch` based code we will test the structural optimization with `torch.optim` optimizers.

### MBB Beam

The first problem that we will run is with the **MBB BEAM**.

In [4]:
def constrained_structural_optimization_function(model, ke, args, designs, losses):
    """
    Combined function for PyGranso for the structural optimization
    problem. The inputs will be the model that reparameterizes x as a function
    of a neural network. V0 is the initial volume, K is the global stiffness
    matrix and F is the forces that are applied in the problem.
    """
    # Initialize the model
    # In my version of the model it follows the similar behavior of the
    # tensorflow repository and only needs None to initialize and output
    # a first value of x
    logits = model(None)

    # kwargs for displacement
    kwargs = dict(
        penal=torch.tensor(args["penal"]),
        e_min=torch.tensor(args["young_min"]),
        e_0=torch.tensor(args["young"]),
    )
    x_phys = torch.sigmoid(logits)

    # Calculate the forces
    forces = topo_physics.calculate_forces(x_phys, args)

    # Calculate the u_matrix
    u_matrix, _ = topo_physics.sparse_displace(
        x_phys, ke, args, forces, args["freedofs"], args["fixdofs"], **kwargs
    )

    # Calculate the compliance output
    compliance_output, _, _ = topo_physics.compliance(x_phys, u_matrix, ke, args, **kwargs)

    # The loss is the sum of the compliance
    f = torch.abs(torch.sum(compliance_output))

    # Run this problem with no inequality constraints
    ci = None

    # Run this problem with no equality constraints
    ce = pygransoStruct()
    ce.c1 = 5e2 * (torch.mean(x_phys) - args['volfrac'])

    # Append updated physical density designs
    designs.append(
        x_phys
    )  # noqa

    return f, ci, ce

In [ ]:
# Identify the problem
problem = problems.PROBLEMS_BY_NAME['multistory_building_32x64_0.5']

# Get the problem args
args = topo_api.specified_task(problem)
cnn_kwargs = None

# Trials
trials = []

for seed in range(41, 70):
    torch.random.manual_seed(seed)

    # Initialize the CNN Model
    if cnn_kwargs is not None:
        cnn_model = models.CNNModel(args, **cnn_kwargs)
    else:
        cnn_model = models.CNNModel(args)

    # Put the cnn model in training mode
    cnn_model.train()

    # Create the stiffness matrix
    ke = topo_physics.get_stiffness_matrix(
        young=args["young"],
        poisson=args["poisson"],
    )

    # Create the combined function and structural optimization
    # setup
    # Save the physical density designs & the losses
    designs = []
    losses = []
    # Combined function
    comb_fn = lambda model: constrained_structural_optimization_function(  # noqa
        model, ke, args, designs, losses
    )

    # Initalize the pygranso options
    opts = pygransoStruct()

    # Set the device
    opts.torch_device = torch.device('cpu')

    # Setup the intitial inputs for the solver
    nvar = getNvarTorch(cnn_model.parameters())
    opts.x0 = (
        torch.nn.utils.parameters_to_vector(cnn_model.parameters())
        .detach()
        .reshape(nvar, 1)
    )

    # Additional pygranso options
    opts.limited_mem_size = 20
    opts.double_precision = True
    opts.mu0 = 1.0
    opts.maxit = 500
    opts.print_frequency = 10
    opts.stat_l2_model = False
    opts.viol_eq_tol = 1e-6
    opts.opt_tol = 1e-4
    
    mHLF_obj = utils.HaltLog()
    halt_log_fn, get_log_fn = mHLF_obj.makeHaltLogFunctions(opts.maxit)

    #  Set PyGRANSO's logging function in opts
    opts.halt_log_fn = halt_log_fn

    # Main algorithm with logging enabled.
    soln = pygranso(var_spec=cnn_model, combined_fn=comb_fn, user_opts=opts)

    # GET THE HISTORY OF ITERATES
    # Even if an error is thrown, the log generated until the error can be
    # obtained by calling get_log_fn()
    log = get_log_fn()
    
    # Final structure
    designs_indexes = (pd.Series(log.fn_evals).cumsum() - 1).values.tolist()
    final_designs = [designs[i] for i in designs_indexes]
    
    trials.append((soln.final.f, pd.Series(log.f), final_designs))

/Users/ryandevera/data-science/tesorio/NCVX-Neural-Structural-Optimization/models.py:70: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  self.h = args["nely"] // total_resize
/Users/ryandevera/data-science/tesorio/NCVX-Neural-Structural-Optimization/models.py:71: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  self.w = args["nelx"] // tot



╔═════ QP SOLVER NOTICE ════════════════════════════════════════════════════════════════════════╗
║  PyGRANSO requires a quadratic program (QP) solver that has a quadprog-compatible interface,  ║
║  the default is osqp. Users may provide their own wrapper for the QP solver.                  ║
║  To disable this notice, set opts.quadprog_info_msg = False                                   ║
╚═══════════════════════════════════════════════════════════════════════════════════════════════╝
═════════════════════════════════════════════════════════════════════════════════════════════════════════════════╗
PyGRANSO: A PyTorch-enabled port of GRANSO with auto-differentiation                                             ║ 
Version 1.2.0                                                                                                    ║ 
Licensed under the AGPLv3, Copyright (C) 2021-2022 Tim Mitchell and Buyun Liang                                  ║ 
═════════════════════════════════════════════

/Users/ryandevera/.pyenv/versions/3.9.11/lib/python3.9/site-packages/pygranso/private/bfgsHessianInverseLimitedMem.py:237: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matricesor `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at  /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/TensorShape.cpp:2985.)
  alpha[j,:]  = self.rho[0,j] * (self.S[:,j].T  @ q)


  10 ║ 0.282430 │  19.9584699140 ║  70.5642652505 ║   -  │ 0.029037 ║ S  │     1 │ 1.000000 ║     1 │ 0.005158   ║ 
  20 ║ 0.228768 │  14.4635190842 ║  63.1651584254 ║   -  │ 0.013357 ║ S  │     1 │ 1.000000 ║     1 │ 0.002535   ║ 
  30 ║ 0.185302 │  10.9991942360 ║  59.0647899302 ║   -  │ 0.054369 ║ S  │     2 │ 0.500000 ║     1 │ 0.076891   ║ 
  40 ║ 0.185302 │  10.5328791172 ║  56.7316702798 ║   -  │ 0.020386 ║ S  │     1 │ 1.000000 ║     1 │ 0.003505   ║ 
  50 ║ 0.135085 │  7.54859170906 ║  55.6982822580 ║   -  │ 0.024580 ║ S  │     1 │ 1.000000 ║     1 │ 0.012203   ║ 
  60 ║ 0.135085 │  7.35319679456 ║  54.3194451505 ║   -  │ 0.015445 ║ S  │     3 │ 0.250000 ║     1 │ 0.029899   ║ 
  70 ║ 0.098477 │  5.29094956931 ║  53.6140694277 ║   -  │ 0.011192 ║ S  │     2 │ 0.500000 ║     1 │ 0.019311   ║ 
  80 ║ 0.098477 │  5.24377057806 ║  53.1067383628 ║   -  │ 0.013974 ║ S  │     4 │ 0.125000 ║     1 │ 0.009319   ║ 
  90 ║ 0.064611 │  3.35340711123 ║  51.8851864082 ║   -  │ 0.001063 ║ S 

  90 ║ 0.020276 │  1.13379022413 ║  55.8428037215 ║   -  │ 0.001546 ║ S  │     5 │ 0.062500 ║     1 │ 0.006089   ║ 
 100 ║ 0.020276 │  1.12596173164 ║  55.4513159922 ║   -  │ 0.001655 ║ S  │     3 │ 1.500000 ║     1 │ 0.001842   ║ 
 110 ║ 0.007070 │  0.39127469661 ║  55.3170511609 ║   -  │ 2.02e-04 ║ S  │     1 │ 1.000000 ║     1 │ 9.76e-04   ║ 
 120 ║ 0.003043 │  0.16767264640 ║  54.9943923927 ║   -  │ 3.11e-04 ║ S  │     9 │ 0.003906 ║     1 │ 0.002677   ║ 
 130 ║ 0.003043 │  0.16687619680 ║  54.7749716805 ║   -  │ 1.82e-04 ║ S  │     1 │ 1.000000 ║     1 │ 0.002332   ║ 
 140 ║ 0.003043 │  0.16667858894 ║  54.7474816270 ║   -  │ 6.82e-05 ║ S  │     8 │ 0.023438 ║     3 │ 4.70e-04   ║ 
 150 ║ 0.001061 │  0.05805446400 ║  54.6908299721 ║   -  │ 2.11e-05 ║ S  │    17 │ 4.58e-05 ║     1 │ 0.001280   ║ 
═════╩═══════════════════════════╩════════════════╩═════════════════╩═══════════════════════╩════════════════════╣
Optimization results:                                                    

...



╔═════ QP SOLVER NOTICE ════════════════════════════════════════════════════════════════════════╗
║  PyGRANSO requires a quadratic program (QP) solver that has a quadprog-compatible interface,  ║
║  the default is osqp. Users may provide their own wrapper for the QP solver.                  ║
║  To disable this notice, set opts.quadprog_info_msg = False                                   ║
╚═══════════════════════════════════════════════════════════════════════════════════════════════╝
═════════════════════════════════════════════════════════════════════════════════════════════════════════════════╗
PyGRANSO: A PyTorch-enabled port of GRANSO with auto-differentiation                                             ║ 
Version 1.2.0                                                                                                    ║ 
Licensed under the AGPLv3, Copyright (C) 2021-2022 Tim Mitchell and Buyun Liang                                  ║ 
═════════════════════════════════════════════

  20 ║ 0.121577 │  8.89092901588 ║  72.4957294310 ║   -  │ 0.077141 ║ S  │     1 │ 1.000000 ║     1 │ 0.047494   ║ 
  30 ║ 0.052335 │  3.49351612442 ║  66.7001482139 ║   -  │ 0.002780 ║ S  │     5 │ 0.062500 ║     1 │ 0.144250   ║ 
  40 ║ 0.016423 │  1.08080106524 ║  65.7544832245 ║   -  │ 9.02e-04 ║ S  │     5 │ 0.062500 ║     1 │ 0.010879   ║ 
  50 ║ 0.016423 │  1.07675982542 ║  65.4876159277 ║   -  │ 0.001243 ║ S  │     3 │ 0.250000 ║     1 │ 0.020005   ║ 
  60 ║ 0.014781 │  0.95560730858 ║  64.5631966184 ║   -  │ 0.001306 ║ S  │     3 │ 0.250000 ║     1 │ 0.047317   ║ 
  70 ║ 0.005726 │  0.36917727599 ║  64.4471202833 ║   -  │ 1.26e-04 ║ S  │     4 │ 0.125000 ║     1 │ 0.015366   ║ 
  80 ║ 0.005726 │  0.36761307391 ║  64.1935982726 ║   -  │ 1.38e-05 ║ S  │     1 │ 1.000000 ║     1 │ 0.012267   ║ 
  90 ║ 0.001997 │  0.12814505168 ║  64.1760744733 ║   -  │ 6.09e-06 ║ S  │     9 │ 0.011719 ║     1 │ 0.004596   ║ 
 100 ║ 0.001997 │  0.12807982525 ║  64.1426877803 ║   -  │ 7.52e-06 ║ S 

 210 ║ 2.43e-04 │  0.01451069803 ║  59.7762525113 ║   -  │ 4.59e-08 ║ S  │    15 │ 0.002502 ║     2 │ 4.81e-05   ║ 
═════╩═══════════════════════════╩════════════════╩═════════════════╩═══════════════════════╩════════════════════╣
Optimization results:                                                                                            ║ 
F = final iterate, B = Best (to tolerance), MF = Most Feasible                                                   ║ 
═════╦═══════════════════════════╦════════════════╦═════════════════╦═══════════════════════╦════════════════════╣
   F ║          │                ║  59.7762525113 ║   -  │ 4.59e-08 ║    │       │          ║       │            ║ 
   B ║          │                ║  59.7762171930 ║   -  │ 6.20e-07 ║    │       │          ║       │            ║ 
  MF ║          │                ║  59.7944420757 ║   -  │ 1.30e-08 ║    │       │          ║       │            ║ 
═════╩═══════════════════════════╩════════════════╩═════════════════╩═════

  10 ║ 0.313811 │  21.2259050212 ║  67.6294975699 ║   -  │ 0.003052 ║ S  │     1 │ 1.000000 ║     1 │ 0.007105   ║ 
  20 ║ 0.254187 │  13.7859655314 ║  54.1537805291 ║   -  │ 0.020801 ║ S  │     1 │ 1.000000 ║     1 │ 0.102027   ║ 
  30 ║ 0.088629 │  4.54711338582 ║  51.1763908601 ║   -  │ 0.011382 ║ S  │     4 │ 0.125000 ║     1 │ 0.809722   ║ 
  40 ║ 0.088629 │  4.42498685993 ║  49.8002282609 ║   -  │ 0.011223 ║ S  │     3 │ 0.750000 ║     1 │ 0.030581   ║ 
  50 ║ 0.088629 │  4.30508425886 ║  48.4351608642 ║   -  │ 0.012306 ║ S  │     7 │ 0.015625 ║     1 │ 0.022116   ║ 
  60 ║ 0.052335 │  2.49291460420 ║  47.5819191821 ║   -  │ 0.002726 ║ S  │     5 │ 0.062500 ║     1 │ 0.018415   ║ 
  70 ║ 0.052335 │  2.47343139235 ║  47.1448944422 ║   -  │ 0.006115 ║ S  │    10 │ 0.001953 ║     1 │ 0.276926   ║ 
  80 ║ 0.052335 │  2.43862765039 ║  46.4706423654 ║   -  │ 0.006598 ║ S  │     9 │ 0.011719 ║     1 │ 0.077147   ║ 
  90 ║ 0.030903 │  1.41908337810 ║  45.7586156426 ║   -  │ 0.004998 ║ S 



╔═════ QP SOLVER NOTICE ════════════════════════════════════════════════════════════════════════╗
║  PyGRANSO requires a quadratic program (QP) solver that has a quadprog-compatible interface,  ║
║  the default is osqp. Users may provide their own wrapper for the QP solver.                  ║
║  To disable this notice, set opts.quadprog_info_msg = False                                   ║
╚═══════════════════════════════════════════════════════════════════════════════════════════════╝
═════════════════════════════════════════════════════════════════════════════════════════════════════════════════╗
PyGRANSO: A PyTorch-enabled port of GRANSO with auto-differentiation                                             ║ 
Version 1.2.0                                                                                                    ║ 
Licensed under the AGPLv3, Copyright (C) 2021-2022 Tim Mitchell and Buyun Liang                                  ║ 
═════════════════════════════════════════════

In [ ]:
soln.final.f

In [ ]:
best_trial = sorted(trials)[0]
best_trial[0]

In [ ]:
# Plot the loss
ax = best_trial[1].cummin().plot(figsize=(10, 7), marker='*')
ax.grid()

In [ ]:
from scipy.ndimage import gaussian_filter

pygranso_structure = best_trial[2][-1].detach().numpy()

# Plot the two structures together
fig, ax1 = plt.subplots(1, 1, figsize=(6, 4))

# pygranso
ax1.imshow(pygranso_structure, cmap='Greys')
ax1.grid()
ax1.set_xlabel('x')
ax1.set_ylabel('y')
ax1.set_title('Multi-story Building 32x64 Density 0.5')
fig.tight_layout()